# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [29]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    #global nhl_df
    cities_nhl = cities.copy()
    cities_nhl = cities_nhl[["Metropolitan area", "Population (2016 est.)[8]", "NHL"]]
    cities_nhl = cities_nhl.replace("(\[.*\])", "", regex = True)\
                .replace("—", np.nan, regex=True).dropna()

    def camel_case_split(str):                                         # camel_case_split function from
        return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)   # https://www.geeksforgeeks.org/python-split-camelcase-string-to-individual-strings/

    cities_nhl["NHL"] = cities_nhl["NHL"].apply(lambda x: camel_case_split(x))
    cities_nhl = cities_nhl["NHL"].apply(pd.Series).merge(cities_nhl, right_index = True, left_index = True)
    cities_nhl = cities_nhl.melt(id_vars = ["Metropolitan area", "NHL", "Population (2016 est.)[8]"], value_name = "team_name")
    cities_nhl = cities_nhl.dropna().drop(["NHL", "variable"], axis=1)
    cities_nhl = cities_nhl.set_index("team_name")
    
    nhl_df2 = nhl_df.copy()[["team", "W", "L", "year"]]
    nhl_df2 = nhl_df2[nhl_df2["year"]==2018].drop([0,9,18,26],axis=0)
    nhl_df2["team"].replace("\*", "", regex = True, inplace = True)
    nhl_df2["team_name"] = nhl_df2["team"].str.split().str[-1]
    nhl_df2 = nhl_df2.set_index("team_name")
    
    nhl_merged = pd.merge(cities_nhl, nhl_df2, left_index = True, right_index = True)
    nhl_merged["Win Loss Ratio"] = nhl_merged["W"].astype(int) / (nhl_merged["W"].astype(int) + nhl_merged["L"].astype(int))
    nhl_merged["Population (2016 est.)[8]"] = nhl_merged["Population (2016 est.)[8]"].astype(int)
    
    #raise NotImplementedError()
    
    population_by_region = nhl_merged.groupby("Metropolitan area")["Population (2016 est.)[8]"].mean() # pass in metropolitan area population from cities
    win_loss_by_region = nhl_merged.groupby("Metropolitan area")["Win Loss Ratio"].mean() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [30]:
nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [31]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    def camel_case_split(str):
        return re.findall(r'[A-Z\d](?:[a-z\d]+|[A-Z]*(?=[A-Z]|$))', str)
    
    cities2 = cities[['Metropolitan area', 'NBA', 'Population (2016 est.)[8]']]
    cities2 = cities2.replace({'NBA': '\[.*\]'}, {'NBA' : ''}, regex = True)\
                .replace({'NBA': '—'}, {'NHL' : np.nan}, regex = True)\
                .dropna()

    cities2['Split'] = cities2['NBA'].apply(lambda x: camel_case_split(x))
    cities2 = cities2.Split.apply(pd.Series) \
                .merge(cities2, right_index = True, left_index = True) \
                .melt(id_vars = ['Metropolitan area', 'Split', 'NBA', 'Population (2016 est.)[8]'], value_name = "Team")
    cities2 = cities2.dropna().drop(['Split', 'NBA', 'variable'], axis=1).set_index('Team')
    
    #Clean up NBA
    nba_df2 = nba_df[(~nba_df.team.str.contains('Division')) & (nba_df.year == 2018)][['team','W','L','year']]
    nba_df2 = nba_df2.replace(to_replace = r'\*?\s\(.*\)', value = '', regex = True)
    nba_df2['team_name'] = nba_df2['team'].str.split().str[-1]
    nba_df2 = nba_df2.set_index('team_name')
    
    #Merge on truncated team name
    merged = nba_df2.merge(cities2, how='left', left_index = True, right_index=True)
    merged['Win Loss Ratio']= merged['W'].astype('int')/(merged['W'].astype('int')+merged['L'].astype('int'))
    merged['Population (2016 est.)[8]'] = merged['Population (2016 est.)[8]'].astype('int')
    
    #raise NotImplementedError()
    
    population_by_region = merged.groupby("Metropolitan area")["Population (2016 est.)[8]"].mean() # pass in metropolitan area population from cities
    win_loss_by_region = merged.groupby("Metropolitan area")["Win Loss Ratio"].mean() # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [32]:
nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [33]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    #global mlb_df
    cities_mlb = cities.copy()
    cities_mlb = cities_mlb[["Metropolitan area", "Population (2016 est.)[8]", "MLB"]]
    cities_mlb = cities_mlb.replace("(\[.*\])", "", regex = True)\
                .replace("—", np.nan, regex=True).dropna()

    def camel_case_split(str):                                         # camel_case_split function from
        return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)   # https://www.geeksforgeeks.org/python-split-camelcase-string-to-individual-strings/

    cities_mlb["MLB"] = cities_mlb["MLB"].apply(lambda x: camel_case_split(x))
    cities_mlb = cities_mlb["MLB"].apply(pd.Series).merge(cities_mlb, right_index = True, left_index = True)
    cities_mlb = cities_mlb.melt(id_vars = ["Metropolitan area", "MLB", "Population (2016 est.)[8]"], value_name = "team_name")
    cities_mlb = cities_mlb.dropna().drop(["MLB", "variable"], axis=1)
    cities_mlb["team_name"][7] = "Red Sox"
    cities_mlb["team_name"][13] = "Blue Jays"
    cities_mlb["team_name"][32] = "White Sox"
    cities_mlb.drop(36, axis=0, inplace=True)
    cities_mlb.drop(42, axis=0, inplace=True)
    cities_mlb.drop(61, axis=0, inplace=True)
    
    cities_mlb = cities_mlb.set_index("team_name")
    
    mlb_df2 = mlb_df.copy()[["team", "W", "L", "year"]]
    mlb_df2 = mlb_df2[mlb_df2["year"]==2018]
    mlb_df2["team_name"] = mlb_df2["team"].str.split().str[-1]
    mlb_df2["team_name"][0] = "Red Sox"
    mlb_df2["team_name"][3] = "Blue Jays"
    mlb_df2["team_name"][8] = "White Sox"
    mlb_df2 = mlb_df2.set_index("team_name")
    
    mlb_merged = pd.merge(cities_mlb, mlb_df2, left_index = True, right_index = True)
    mlb_merged["Win Loss Ratio"] = mlb_merged["W"].astype(int) / (mlb_merged["W"].astype(int) + mlb_merged["L"].astype(int))
    mlb_merged["Population (2016 est.)[8]"] = mlb_merged["Population (2016 est.)[8]"].astype(int)
    
    #raise NotImplementedError()
    
    population_by_region = mlb_merged.groupby("Metropolitan area")["Population (2016 est.)[8]"].mean() # pass in metropolitan area population from cities
    win_loss_by_region = mlb_merged.groupby("Metropolitan area")["Win Loss Ratio"].mean() # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [34]:
mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [35]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    #global nfl_df
    cities_nfl = cities.copy()
    cities_nfl = cities_nfl[["Metropolitan area", "Population (2016 est.)[8]", "NFL"]]
    cities_nfl = cities_nfl.replace("(\[.*\])", "", regex = True)\
                .replace("—", np.nan, regex=True).dropna()

    def camel_case_split(str):                                         # camel_case_split function from
        return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)   # https://www.geeksforgeeks.org/python-split-camelcase-string-to-individual-strings/

    cities_nfl["NFL"] = cities_nfl["NFL"].apply(lambda x: camel_case_split(x))
    cities_nfl = cities_nfl["NFL"].apply(pd.Series).merge(cities_nfl, right_index = True, left_index = True)
    cities_nfl = cities_nfl.melt(id_vars = ["Metropolitan area", "NFL", "Population (2016 est.)[8]"], value_name = "team_name")
    cities_nfl = cities_nfl.dropna().drop(["NFL", "variable"], axis=1)
    cities_nfl = cities_nfl.set_index("team_name")
    
    nfl_df2 = nfl_df.copy()[["team", "W", "L", "year"]]
    nfl_df2 = nfl_df2[nfl_df2["year"]==2018].drop([0,5,10,15,20,25,30,35],axis=0)
    nfl_df2["team"].replace("\*", "", regex = True, inplace = True)
    nfl_df2["team"].replace("\+", "", regex = True, inplace = True)
    nfl_df2["team_name"] = nfl_df2["team"].str.split().str[-1]
    nfl_df2 = nfl_df2.set_index("team_name")
    
    nfl_merged = pd.merge(cities_nfl, nfl_df2, left_index = True, right_index = True)
    nfl_merged["Win Loss Ratio"] = nfl_merged["W"].astype(int) / (nfl_merged["W"].astype(int) + nfl_merged["L"].astype(int))
    nfl_merged["Population (2016 est.)[8]"] = nfl_merged["Population (2016 est.)[8]"].astype(int)
    
    #raise NotImplementedError()
    
    population_by_region = nfl_merged.groupby("Metropolitan area")["Population (2016 est.)[8]"].mean() # pass in metropolitan area population from cities
    win_loss_by_region = nfl_merged.groupby("Metropolitan area")["Win Loss Ratio"].mean() # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [36]:
nfl_correlation()

0.004922112149349393

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values